Nernay Kumar
Intern Id 
INT2026-9021

In [ ]:
import numpy as np
import pandas as pd


In [6]:
# Load the saved messy CSV
from pathlib import Path

messy_csv_path = Path(r"C:\Users\My-Pc\Desktop\cepiaLabIntern\week1\day1\interns_ai_ml_assignments\week-1\day-3-assignment\customer_churn_messy.csv")

messy_loaded = pd.read_csv(messy_csv_path)
print(messy_loaded.head())
print(messy_loaded.shape)

   CustomerID   Age  Gender  Tenure  Usage Frequency  Support Calls  \
0         2.0  30.0  FEMALE    39.0             14.0            5.0   
1         3.0  65.0       f    49.0              1.0           10.0   
2         4.0  55.0  female    14.0              4.0            6.0   
3         5.0  58.0    MALE    38.0             21.0            7.0   
4         6.0  23.0       m    32.0             20.0            5.0   

   Payment Delay Subscription Type Contract Length  Total Spend  \
0           18.0          Standard          Annual        932.0   
1            8.0             Basic         Monthly        557.0   
2           18.0             Basic       Quarterly        185.0   
3            7.0          Standard         Monthly        396.0   
4            8.0             Basic         Monthly        617.0   

   Last Interaction  Churn  
0              17.0    1.0  
1               6.0    1.0  
2               3.0    1.0  
3              29.0    1.0  
4              20.0    1.

In [10]:
# Basic checks on messy dataset
messy_df = messy_loaded.copy()

print("Missing values per column:\n", messy_df.isna().sum().sort_values(ascending=False))
print("Duplicate rows:", messy_df.duplicated().sum())
print("Gender values (sample):\n", messy_df["Gender"].value_counts().head())
print("Total Spend range:", messy_df["Total Spend"].min(), "to", messy_df["Total Spend"].max())

Missing values per column:
 Age                  30860
CustomerID               1
Gender                   1
Tenure                   1
Usage Frequency          1
Support Calls            1
Payment Delay            1
Subscription Type        1
Contract Length          1
Total Spend              1
Last Interaction         1
Churn                    1
dtype: int64
Duplicate rows: 8378
Gender values (sample):
 Gender
male    51236
m       51117
Male    51027
M       50992
MALE    50940
Name: count, dtype: int64
Total Spend range: 100.0 to 19998.4


In [11]:
# Cleaning steps
clean_df = messy_df.copy()

# 1) Handle missing Age values (fill with median)
age_median = clean_df["Age"].median()
clean_df["Age"] = clean_df["Age"].fillna(age_median)

# 2) Standardize Gender values
clean_df["Gender"] = clean_df["Gender"].astype(str).str.strip().str.lower()
clean_df["Gender"] = clean_df["Gender"].replace({
    "m": "male",
    "male": "male",
    "f": "female",
    "female": "female"
})

# 3) Remove duplicate rows
clean_df = clean_df.drop_duplicates()

# 4) Handle salary outliers (Total Spend) using IQR clipping
Q1 = clean_df["Total Spend"].quantile(0.25)
Q3 = clean_df["Total Spend"].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
clean_df["Total Spend"] = clean_df["Total Spend"].clip(lower, upper)

print("Cleaning done.")
print("Rows after cleaning:", len(clean_df))

Cleaning done.
Rows after cleaning: 441272


In [15]:
# Verify after cleaning
print("Missing values after cleaning:\n", clean_df.isna().sum().sort_values(ascending=False))
print("Duplicate rows after cleaning:", clean_df.duplicated().sum())
print("Gender values after cleaning:\n", clean_df["Gender"].value_counts())
print("Total Spend range after cleaning:", clean_df["Total Spend"].min(), "to", clean_df["Total Spend"].max())

Missing values after cleaning:
 CustomerID           1
Tenure               1
Support Calls        1
Usage Frequency      1
Contract Length      1
Total Spend          1
Payment Delay        1
Subscription Type    1
Last Interaction     1
Churn                1
Gender               0
Age                  0
dtype: int64
Duplicate rows after cleaning: 0
Gender values after cleaning:
 Gender
male      250492
female    190779
nan            1
Name: count, dtype: int64
Total Spend range after cleaning: 100.0 to 1361.7375000000002


In [13]:
# Save cleaned dataset
clean_output_path = Path(r"C:\Users\My-Pc\Desktop\cepiaLabIntern\week1\day1\interns_ai_ml_assignments\week-1\day-3-assignment\customer_churn_cleaned.csv")
clean_df.to_csv(clean_output_path, index=False)
print("Saved cleaned dataset to:", clean_output_path)

Saved cleaned dataset to: C:\Users\My-Pc\Desktop\cepiaLabIntern\week1\day1\interns_ai_ml_assignments\week-1\day-3-assignment\customer_churn_cleaned.csv


In [ ]:
# Full clean: fix "nan" string in Gender and handle remaining missing values

# Convert literal "nan" string back to NaN
clean_df["Gender"] = clean_df["Gender"].replace("nan", np.nan)

# Fill missing Gender with mode
if clean_df["Gender"].isna().any():
    clean_df["Gender"] = clean_df["Gender"].fillna(clean_df["Gender"].mode()[0])

# Drop any remaining rows with missing values (if any)
clean_df = clean_df.dropna()

print("After full clean:")
print(clean_df.isna().sum().sort_values(ascending=False))
print("Rows after full clean:", len(clean_df))

After full clean:
CustomerID           0
Age                  0
Gender               0
Tenure               0
Usage Frequency      0
Support Calls        0
Payment Delay        0
Subscription Type    0
Contract Length      0
Total Spend          0
Last Interaction     0
Churn                0
dtype: int64
Rows after full clean: 441271


In [ ]:
# Save the fully cleaned dataset as a new file
final_clean_path = Path(r"C:\Users\My-Pc\Desktop\cepiaLabIntern\week1\day1\interns_ai_ml_assignments\week-1\day-3-assignment\customer_churn_cleaned_final.csv")
clean_df.to_csv(final_clean_path, index=False)
print("Saved fully cleaned dataset to:", final_clean_path)

Saved fully cleaned dataset to: C:\Users\My-Pc\Desktop\cepiaLabIntern\week1\day1\interns_ai_ml_assignments\week-1\day-3-assignment\customer_churn_cleaned_final.csv
